In [1]:
import sys
import pandas as pd
import util_mwb
print(sys.version)
print(pd.__version__) # Must be 0.24.1 for IntegerArray (no longer used)

3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0]
0.25.2


In [2]:
# "Original" df5 version from Laritza
csl = pd.read_csv('../data/csl/MFM_data5.csv', index_col=0)

In [3]:
pd.options.display.max_columns = 30
csl.head()

,MomID,Sitenum,Momdeath,MomICU,Bloodproduct,Posttransfus,onepregflag,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,AB_peni,...,Activeherpes,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,transfus_yes,high_EBLoss,high_Gravidity,high_MomLOS,high_height
1,b'41-00002',b'41',1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,38,1,1,0,1,1,1,0,0,1,0,2,3,2.0
2,b'41-00003',b'41',1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,39,1,1,0,1,1,1,0,0,1,0,3,2,2.0
3,b'41-00004',b'41',1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,39,1,1,0,1,1,1,0,0,1,0,4,1,3.0
4,b'41-00005',b'41',1,1,1,2,1,1,1,1,1,1,1,1,1,...,1,38,1,1,0,1,1,1,0,0,2,0,3,2,2.0
5,b'41-00006',b'41',1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,39,1,1,0,1,1,1,0,0,1,0,1,2,3.0


In [5]:
csl['high_Age'].value_counts()

0    162006
3     17242
1      5972
2       193
Name: high_Age, dtype: int64

In [4]:
# Removed extraneous characters from strings; had to use .loc[row, col] to avoid warning
csl.loc[:,'MomID'] = csl['MomID'].str.replace('b','')
csl.loc[:,'MomID'] = csl['MomID'].str.replace("'","")
csl.loc[:,'Sitenum'] = csl['Sitenum'].str.replace('b','')
csl.loc[:,'Sitenum'] = csl['Sitenum'].str.replace("'","")

In [5]:
# Calculate 'MomNearMiss' by OR-ing 'Momdeath' and 'MomICU'
csl.loc[(csl['Momdeath'] == 2) | (csl['MomICU'] == 2), 'MomNearMiss'] = 2
csl['MomNearMiss'].fillna(1, inplace=True)
csl['MomNearMiss'] = csl['MomNearMiss'].astype('Int8')
csl.head()

,MomID,Sitenum,Momdeath,MomICU,Bloodproduct,Posttransfus,onepregflag,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,AB_peni,...,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,transfus_yes,high_EBLoss,high_Gravidity,high_MomLOS,high_height,MomNearMiss
1,41-00002,41,1,1,1,1,1,1,1,1,1,1,1,1,1,...,38,1,1,0,1,1,1,0,0,1,0,2,3,2.0,1
2,41-00003,41,1,1,1,1,1,1,1,1,1,1,1,1,1,...,39,1,1,0,1,1,1,0,0,1,0,3,2,2.0,1
3,41-00004,41,1,1,1,1,1,1,1,1,1,1,1,1,1,...,39,1,1,0,1,1,1,0,0,1,0,4,1,3.0,1
4,41-00005,41,1,1,1,2,1,1,1,1,1,1,1,1,1,...,38,1,1,0,1,1,1,0,0,2,0,3,2,2.0,1
5,41-00006,41,1,1,1,1,1,1,1,1,1,1,1,1,1,...,39,1,1,0,1,1,1,0,0,1,0,1,2,3.0,1


In [6]:
# Can't have any NaN values or many algorithms will puke
for col in csl:
    if csl[col].isna().any():
        print(f'{col} column has NaN values')

In [7]:
from collections import Counter
print(Counter(csl['MomNearMiss']))

Counter({1: 184541, 2: 872})


In [8]:
print(csl['transfus_yes'].value_counts())

1    180243
2      5170
Name: transfus_yes, dtype: int64


In [9]:
csl['onepregflag'].value_counts()

1    185413
Name: onepregflag, dtype: int64

In [10]:
# Need to drop onepregflag since it is constant across all rows
csl.drop('onepregflag', axis=1, inplace=True) 

In [11]:
csl['EBLoss'].value_counts()

350.0    61531
300.0    26808
500.0    16756
250.0    12721
200.0    11710
         ...  
502.0        1
260.0        1
649.0        1
903.0        1
3.0          1
Name: EBLoss, Length: 284, dtype: int64

In [12]:
hiLoss = csl[csl['EBLoss'] >= 1000]
hiLoss['high_EBLoss'].value_counts()

2    5423
3     316
4      21
Name: high_EBLoss, dtype: int64

In [13]:
ty = csl[(csl['transfus_yes'] == 2)]
hloss = csl[csl['EBLoss'] >= 1000]
print(ty['transfus_yes'].value_counts())
hloss['EBLoss'].value_counts().sort_index()

2    5170
Name: transfus_yes, dtype: int64


1000.0      3760
1010.0         2
1050.0         5
1100.0       227
1150.0         3
            ... 
8500.0         1
9999.0         1
10000.0        2
43350.0        1
900150.0       1
Name: EBLoss, Length: 61, dtype: int64

In [14]:
# Calculate 'transLoss' by OR-ing 'transfus_yes' and 'EBLoss' >= 1000
csl.loc[(csl['transfus_yes'] == 2) | (csl['EBLoss'] >= 1000), 'trans_loss'] = 2
csl['trans_loss'].fillna(1, inplace=True)
csl['trans_loss'] = csl['trans_loss'].astype('Int8')
csl['trans_loss'].value_counts()

1    175069
2     10344
Name: trans_loss, dtype: int64

In [15]:
both_df = csl[(csl['transfus_yes'] == 2) & (csl['EBLoss'] >= 1000)]
both_df.shape

(586, 249)

In [16]:
print(csl[csl['transfus_yes'] == 2].shape)
print(csl[csl['EBLoss'] >= 1000].shape)
print(csl[csl['EBLoss'] > 1000].shape)
trans_loss_df = csl[csl['trans_loss'] == 2]
print(trans_loss_df.shape)

(5170, 249)
(5760, 249)
(2000, 249)
(10344, 249)


In [17]:
# "transfus_all" is a new target that includes "transfus_yes" OR "EBLoss >= 1000" OR hysterectomy
print(csl['Hysterectomy'].value_counts())
csl.loc[(csl['transfus_yes'] == 2) | (csl['EBLoss'] >= 1000) | (csl['Hysterectomy'] == 2), 'transfus_all'] = 2
csl['transfus_all'].fillna(1, inplace=True)
csl['transfus_all'] = csl['transfus_all'].astype('Int8')
print(csl['transfus_all'].value_counts())
csl[(csl['transfus_yes'] == 2) & (csl['Hysterectomy'] == 2)].shape

1    185281
2       132
Name: Hysterectomy, dtype: int64
1    175034
2     10379
Name: transfus_all, dtype: int64


(70, 250)

In [18]:
# "transfus_hyster" is new target variable the includes "transfus_yes" OR "Hysterectomy"
print(csl[(csl['transfus_yes'] == 2) | (csl['Hysterectomy'] == 2)].shape)
csl.loc[(csl['Hysterectomy'] == 2) | (csl['transfus_yes'] == 2), 'transfus_hyster'] = 2
csl['transfus_hyster'].fillna(1, inplace=True)
csl['transfus_hyster'] = csl['transfus_hyster'].astype('Int8')
print(csl['transfus_hyster'].value_counts())

(5232, 250)
1    180181
2      5232
Name: transfus_hyster, dtype: int64


In [19]:
# Reorder columns
col_list = list(csl.columns)
new_list = [x for x in col_list if x not in ['transfus_yes','MomNearMiss','trans_loss','EBLoss', 'high_EBLoss','Hysterectomy','transfus_all','transfus_hyster']]
new_list.insert(6, 'transfus_hyster')
new_list.insert(6, 'transfus_all')
new_list.insert(6, 'Hysterectomy')
new_list.insert(6, 'trans_loss')
new_list.insert(6, 'high_EBLoss')
new_list.insert(6, 'EBLoss')
new_list.insert(6, 'transfus_yes')
new_list.insert(4, 'MomNearMiss')
df = pd.DataFrame(csl[new_list])
df.set_index('MomID', inplace=True)
print(df.shape)
df

(185413, 250)


,Sitenum,Momdeath,MomICU,MomNearMiss,Bloodproduct,Posttransfus,transfus_yes,EBLoss,high_EBLoss,trans_loss,Hysterectomy,transfus_all,transfus_hyster,AB_any,AB_ceph,...,Woundinf,Woundsep,Activeherpes,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_MomLOS,high_height
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,41,1,1,1,1,1,1,350.0,0,1,1,1,1,1,1,...,1,1,1,38,1,1,0,1,1,1,0,0,2,3,2.0
41-00003,41,1,1,1,1,1,1,250.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,3,2,2.0
41-00004,41,1,1,1,1,1,1,250.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,4,1,3.0
41-00005,41,1,1,1,1,2,2,350.0,0,2,1,2,2,1,1,...,1,1,1,38,1,1,0,1,1,1,0,0,3,2,2.0
41-00006,41,1,1,1,1,1,1,450.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,1,2,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,52,1,1,1,1,1,1,300.0,0,1,1,1,1,1,1,...,1,1,1,37,1,1,0,1,1,3,0,3,1,2,2.0
52-07248,52,1,1,1,1,1,1,100.0,0,1,1,1,1,1,1,...,1,1,1,24,1,1,1,1,1,3,0,0,2,2,3.0
52-07249,52,1,2,2,2,1,2,2000.0,3,2,1,2,2,1,1,...,1,1,1,28,1,1,1,1,1,3,0,0,3,3,2.0


In [22]:
# CSL_d5_targets contains all d5 dataset with all possible targets included.
# It is intended to be subsetted since many of the targest and variables overlap and 
# would cause multicollinearity if this file were used in predictions.
import os
#filePath = '../data/csl/CSL_d5_targets.csv'
filePath = '../data/csl/CSL_d5_targets_2020-08-21.csv'
# Note: need to comment out the following line to overwrite the file.
if not os.path.exists(filePath):
    df.to_csv(filePath)